In [3]:
import os
import boto
import hashlib
from boto.s3.connection import S3Connection
    
aws_key_id = settings.AWS_ACCESS_KEY_ID
aws_secret_key = settings.AWS_SECRET_ACCESS_KEY
aws_host = settings.AWS_S3_HOST

conn = S3Connection(
    aws_access_key_id=settings.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=settings.AWS_SECRET_ACCESS_KEY,
    is_secure=False, 
    port=None, 
    proxy=None, 
    proxy_port=None, 
    proxy_user=None, 
    proxy_pass=None, 
    host=settings.AWS_S3_HOST,
    debug=0, 
    https_connection_factory=None, 
    calling_format='boto.s3.connection.SubdomainCallingFormat', 
    path='/', 
    provider='aws', 
    #bucket_class=<class 'boto.s3.bucket.Bucket'>, 
    security_token=None, 
    suppress_consec_slashes=True, 
    anon=False, 
    validate_certs=None, 
    profile_name=None
    )
static = conn.get_bucket(settings.AWS_STORAGE_BUCKET_NAME)

In [5]:
def hashfile(fname, algorithm=hashlib.md5, blocksize=65536):
    """Hexadecimal hash of a file"""
    hasher = algorithm()
    with open(fname, 'rb') as source:
        buffer = source.read(blocksize)
        while len(buffer) > 0:
            hasher.update(buffer)
            buffer = source.read(blocksize)
    return hasher.hexdigest()

In [4]:
for key in static.get_all_keys(max_keys=3):
    print(key.BufferSize, key.size, key.name, key.etag)

key = static.get_key('media/pdf/universitas_2015-28.pdf')
fname = os.path.join('/tmp/', os.path.split(key.name)[1])
key.get_contents_to_filename(fname)

8192 2943229 arkiv/2002/01/01-01.pdf "1c18eb039366f4610af2cfbdb524ace4"
8192 780482 arkiv/2002/01/01-02.pdf "5199237591f774d88cbe4535d926dbe5"
8192 1373175 arkiv/2002/01/01-03.pdf "690c9d3ea81993c7369300ef47e6264c"


In [8]:
print(key.etag.strip('"'), hashfile(fname), sep='\n')
print(key.size, os.stat(fname).st_size) 
print((int(boto.utils.parse_ts(key.last_modified).strftime('%s')) - int(os.stat(fname).st_mtime)) % 60 * 60) 

b054063e5099c3c880bc39520764b354
b054063e5099c3c880bc39520764b354
5992292 5992292
0


# Check local staging folder